In [1]:
import pandas as pd
import os
os.chdir("C:\\Siddhartha\\Confidential Documents\\Data Science\\MT 599\\")
dataset=pd.read_csv("MT599.csv")

In [2]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
#import mpld3

In [3]:
# Splitting the dataset into the Training set and Test set
df=pd.DataFrame(dataset["Text"])

In [5]:
stop_words = pd.read_csv("stop word list - V1.csv")
stop_words = stop_words['i'].tolist()

In [7]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [8]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [9]:
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in df["Text"]:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

NameError: name 'data' is not defined

In [10]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('There are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

There are 91302 items in vocab_frame


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words=stop_words,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df["Text"]) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

Wall time: 3.15 s
(1519, 13)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [12]:
terms = tfidf_vectorizer.get_feature_names()

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print(dist)
print

[[-2.22044605e-16 -2.22044605e-16  3.65450932e-01 ...  4.00925445e-01
   4.00925445e-01  4.00925445e-01]
 [-2.22044605e-16 -2.22044605e-16  3.65450932e-01 ...  4.00925445e-01
   4.00925445e-01  4.00925445e-01]
 [ 3.65450932e-01  3.65450932e-01 -2.22044605e-16 ...  7.44911540e-01
   7.44911540e-01  7.44911540e-01]
 ...
 [ 4.00925445e-01  4.00925445e-01  7.44911540e-01 ... -2.22044605e-16
  -2.22044605e-16 -2.22044605e-16]
 [ 4.00925445e-01  4.00925445e-01  7.44911540e-01 ... -2.22044605e-16
  -2.22044605e-16 -2.22044605e-16]
 [ 4.00925445e-01  4.00925445e-01  7.44911540e-01 ... -2.22044605e-16
  -2.22044605e-16 -2.22044605e-16]]


<function print>

In [15]:
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

Wall time: 316 ms


In [16]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [17]:
Revised = { 'Text': df["Text"], 'cluster': clusters,}

Output = pd.DataFrame(Revised, index = [clusters] , columns = ['Text', 'cluster'])

In [18]:
Output['cluster'].value_counts() #number of topics per cluster (clusters from 0 to 9)

1    271
5    234
3    196
2    184
7    152
4    131
0    120
9     92
6     76
8     63
Name: cluster, dtype: int64

In [19]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :15]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print() #add whitespace
    print() #add whitespace
    

Top terms per cluster:

Cluster 0 words: limited, following, any, singapore, securities, per, mt599, hereby, dear, chartered, chartered, amount, account,

Cluster 1 words: hereby, dear,

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if sys.path[0] == '':


 limited, any, following, chartered, securities, account, singapore, per, mt599, chartered, amount,

Cluster 2 words: per, chartered, singapore, amount, chartered, account, following, any, securities, mt599, dear, limited, hereby,

Cluster 3 words: account, amount, singapore, limited, following, mt599, per, securities, any, hereby, dear, chartered, chartered,

Cluster 4 words: mt599, securities, per, account, any, singapore, limited, hereby, following, dear, chartered, chartered, amount,

Cluster 5 words: account, securities, mt599, per, following, chartered, singapore, chartered, limited, dear, any, hereby, amount,

Cluster 6 words: amount, any, account, securities, hereby, following, singapore, per, mt599, limited, dear, chartered, chartered,

Cluster 7 words: singapore, mt599, following, chartered, chartered, account, securities, hereby, limited, any, amount, per, dear,

Cluster 8 words: securities, account, chartered, any, following, per, amount, singapore, mt599, limited, hereby, 